In [1]:
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
import json

In [2]:
holding = pd.read_csv('./data/etf_holding_info.csv')
holding.rename(columns={"symbol": "holding", "name": "holding_name"}, inplace=True)

company_desc = pd.read_csv('./data/company_data.csv')
company_desc.rename(columns={"symbol": "holding", "description": "holding_desc"}, inplace=True)

etf_desc = pd.read_csv('./data/etf_data.csv')
etf_desc.rename(columns={"symbol": "etf", "name": "etf_name", "merged_description": "etf_desc"}, inplace=True)

company_desc = company_desc[['holding', 'holding_desc']]
etf_desc = etf_desc[['etf', 'etf_name', 'etf_desc']]

merged_df = pd.merge(holding, etf_desc, on="etf", how="inner")
final_df = pd.merge(merged_df, company_desc, on="holding", how="inner")
# 개별종목 결측치 제거
final_df = final_df.dropna(subset=['holding_name', 'holding_desc'])
# ETF별 개별종목 중복 제거
final_df = final_df.drop_duplicates(subset=['etf', 'holding']).reset_index(drop=True)

In [4]:
file_path = './data/rewrite_final_v2.json'

with open(file_path, 'r', encoding='utf-8') as f:
    rewritten_descriptions = json.load(f)

final_df['holding_rewritten'] = final_df['holding'].map(rewritten_descriptions)
print("length of final_df: ", len(final_df))
print("final_df is null: ", final_df.isnull().sum())
final_df.head(5)

length of final_df:  156709
final_df is null:  etf                  0
holding              0
holding_name         0
industry             0
etf_name             0
etf_desc             0
holding_desc         0
holding_rewritten    0
dtype: int64


,etf,holding,holding_name,industry,etf_name,etf_desc,holding_desc,holding_rewritten
0,PGF,JPM,JPMorgan Chase & Co,Banks - Diversified,Invesco Financial Preferred ETF,The Invesco Financial Preferred ETF (PGF) seek...,JPMorgan Chase & Co. operates as a financial s...,"This entity operates as a premier, globally sy..."
1,PGF,ALL-PJ,Allstate Corp/The,Insurance - Property & Casualty,Invesco Financial Preferred ETF,The Invesco Financial Preferred ETF (PGF) seek...,"The Allstate Corporation, together with its su...",This entity operates as a major financial inst...
2,PGF,WFC-PC,Wells Fargo & Co,Banks - Diversified,Invesco Financial Preferred ETF,The Invesco Financial Preferred ETF (PGF) seek...,"Wells Fargo & Company, a diversified financial...",This institution serves as a cornerstone withi...
3,PGF,MET-PA,MetLife Inc,Insurance - Life,Invesco Financial Preferred ETF,The Invesco Financial Preferred ETF (PGF) seek...,"MetLife, Inc., a financial services company, p...",This entity is identified through its signific...
4,PGF,MTB-PJ,M&T Bank Corp,Banks - Regional,Invesco Financial Preferred ETF,The Invesco Financial Preferred ETF (PGF) seek...,M&T Bank Corporation operates as a bank holdin...,This equity represents a financial institution...


In [5]:
# --- 1. 각 ETF별 보유 종목 수 계산 ---
print("\nCalculating holdings per ETF...")
etf_holding_counts = final_df.groupby('etf')['holding'].count() 

# --- 2. 조건에 따라 ETF 그룹 분리 --- K=10까지 충족시키기 위함
min_holdings_for_test = 10
etfs_enough_holdings = etf_holding_counts[etf_holding_counts >= min_holdings_for_test].index.tolist()
etfs_few_holdings = etf_holding_counts[etf_holding_counts < min_holdings_for_test].index.tolist()

n_total_etfs = len(etf_holding_counts)
n_enough = len(etfs_enough_holdings)
n_few = len(etfs_few_holdings)
print(f"\nFound {n_total_etfs} unique ETFs.")
print(f" - {n_enough} ETFs with >= {min_holdings_for_test} holdings (candidates for test set)")
print(f" - {n_few} ETFs with < {min_holdings_for_test} holdings")
# 전체 ETF 수가 맞는지 확인
if n_enough + n_few != n_total_etfs:
     print(f"Warning: ETF group counts ({n_enough} + {n_few}) do not sum to total ETFs ({n_total_etfs})!")


# --- 3. 분할 비율 정의 ---
validation_set_ratio = 0.1  # 예: 10% validation
test_set_ratio = 0.2      # 예: 20% test
train_set_ratio = 1.0 - validation_set_ratio - test_set_ratio # 나머지 train (70%)

print(f"\nTarget split ratios: Train={train_set_ratio:.2f}, Validation={validation_set_ratio:.2f}, Test={test_set_ratio:.2f}")
if not np.isclose(train_set_ratio + validation_set_ratio + test_set_ratio, 1.0):
    print("Warning: Defined ratios do not sum to 1.0!")

# --- 4. 목표 세트 크기 계산 (전체 ETF 기준) ---
# 비율에 따라 각 세트에 할당될 ETF의 목표 개수 계산
n_test_target = int(n_total_etfs * test_set_ratio)
n_valid_target = int(n_total_etfs * validation_set_ratio)
# 나머지는 train으로 초기 할당 (나중에 조정될 수 있음)
n_train_target = n_total_etfs - n_test_target - n_valid_target
print(f"Target set sizes (approx): Train={n_train_target}, Validation={n_valid_target}, Test={n_test_target}")

# --- 5. ETF 그룹 섞기 ---
random_seed = 42     # 재현성을 위한 시드
np.random.seed(random_seed)
# 각 그룹을 독립적으로 섞음
shuffled_etfs_enough = np.random.permutation(etfs_enough_holdings)
shuffled_etfs_few = np.random.permutation(etfs_few_holdings)

# --- 6. Test Set 할당 ---
# 보유 종목 수가 충분한 ETF 그룹에서 Test set 목표 크기만큼 할당 시도
print(f"\nAssigning Test Set (target: {n_test_target} ETFs, source: >= {min_holdings_for_test} holdings group)...")
if n_enough < n_test_target:
    # Test set 목표 크기보다 후보 ETF 수가 적으면, 가능한 모든 후보를 test set으로 사용
    print(f"  Warning: Only {n_enough} ETFs available with enough holdings for the test set (target was {n_test_target}). Using all {n_enough}.")
    actual_n_test = n_enough
    test_etf_tickers = shuffled_etfs_enough.tolist() # 사용 가능한 모든 ETF
    remaining_shuffled_etfs_enough = [] # 남는 후보 없음
else:
    # Test set 목표 크기만큼 할당 가능하면, 앞에서부터 할당
    actual_n_test = n_test_target
    test_etf_tickers = shuffled_etfs_enough[:actual_n_test].tolist()
    remaining_shuffled_etfs_enough = shuffled_etfs_enough[actual_n_test:].tolist()

print(f"  Assigned {len(test_etf_tickers)} ETFs to the Test set.")

# --- 7. Train 및 Validation Set 할당 ---
# Test set에 할당되지 않은 나머지 모든 ETF들을 합침
pool_for_train_valid_list = remaining_shuffled_etfs_enough + etfs_few_holdings # list concatenation
pool_for_train_valid = np.array(pool_for_train_valid_list)
np.random.shuffle(pool_for_train_valid) # 합쳐진 풀을 다시 섞음

n_remaining_for_train_valid = len(pool_for_train_valid)
print(f"\nAssigning Train/Validation Sets from remaining {n_remaining_for_train_valid} ETFs...")

# 남은 ETF 풀에서 원래 Train/Validation 비율에 맞춰 실제 할당 크기 계산
n_total_assigned_to_train_valid_target = n_train_target + n_valid_target

if n_remaining_for_train_valid == 0:
     actual_n_valid = 0
     actual_n_train = 0
elif n_total_assigned_to_train_valid_target <= 0: # Train/Valid 목표 합계가 0 이하일 경우
     print("  Warning: Target size for Train+Validation is zero or negative. Assigning all remaining to Train.")
     actual_n_valid = 0
     actual_n_train = n_remaining_for_train_valid # 남은 모든 ETF를 Train으로
else:
    # 원래 목표 비율에 따라 남은 풀 분배
    # 소수점 계산 후 정수로 변환 시 오차 발생 가능하므로 주의
    ratio_valid_in_pool = n_valid_target / n_total_assigned_to_train_valid_target
    actual_n_valid = int(np.round(n_remaining_for_train_valid * ratio_valid_in_pool)) # 반올림 사용 가능
    # 실제 할당 크기가 풀 크기를 넘지 않도록 보정
    actual_n_valid = min(actual_n_valid, n_remaining_for_train_valid)
    actual_n_train = n_remaining_for_train_valid - actual_n_valid

print(f"  Targeting Train={actual_n_train}, Validation={actual_n_valid} from the remaining pool.")

# 실제 분할 수행
train_etf_tickers = pool_for_train_valid[:actual_n_train].tolist()
valid_etf_tickers = pool_for_train_valid[actual_n_train:actual_n_train + actual_n_valid].tolist()

# 최종 할당된 ETF 개수 확인
final_n_train = len(train_etf_tickers)
final_n_valid = len(valid_etf_tickers)
final_n_test = len(test_etf_tickers)

print(f"\nFinal assigned counts: Train={final_n_train}, Validation={final_n_valid}, Test={final_n_test}")
if final_n_train + final_n_valid + final_n_test != n_total_etfs:
     print(f"  Warning: Final counts ({final_n_train + final_n_valid + final_n_test}) do not sum to total ETFs ({n_total_etfs})!")
else:
     print(f"  Total assigned ETFs match total unique ETFs ({n_total_etfs}).")

# 실제 분할 비율 출력
if n_total_etfs > 0:
    print(f"\nActual split ratios: Train={final_n_train/n_total_etfs:.2%}, Validation={final_n_valid/n_total_etfs:.2%}, Test={final_n_test/n_total_etfs:.2%}")


# --- 8. 최종 DataFrame 생성 ---
print("\nCreating final DataFrames...")
train_df = final_df[final_df['etf'].isin(train_etf_tickers)].copy()
valid_df = final_df[final_df['etf'].isin(valid_etf_tickers)].copy()
test_df  = final_df[final_df['etf'].isin(test_etf_tickers)].copy()

# --- 9. 검증 ---
print(f"\nTrain DataFrame shape:      {train_df.shape}")
print(f"Validation DataFrame shape: {valid_df.shape}")
print(f"Test DataFrame shape:       {test_df.shape}")

total_split_rows = len(train_df) + len(valid_df) + len(test_df)
print(f"\nTotal rows in split: {total_split_rows}")
if total_split_rows == len(final_df):
    print(f"Row count matches original DataFrame size ({len(final_df)}).")
else:
    print(f"Warning: Row count ({total_split_rows}) does not match original DataFrame size ({len(final_df)})!")

# Test set의 ETF들이 실제로 조건을 만족하는지 최종 확인
if final_n_test > 0: # Test set이 비어있지 않은 경우에만 확인
    test_etf_actual_counts = final_df[final_df['etf'].isin(test_etf_tickers)].groupby('etf')['holding'].count()
    if (test_etf_actual_counts < min_holdings_for_test).any():
        print(f"\nError: Some ETFs in the final test set have < {min_holdings_for_test} holdings!")
    else:
        print(f"\nVerification successful: All {len(test_etf_tickers)} ETFs in the test set have >= {min_holdings_for_test} holdings.")
elif final_n_test == 0:
     print(f"\nVerification note: Test set is empty (no ETFs met the criteria or test ratio was too small).")


Calculating holdings per ETF...

Found 1315 unique ETFs.
 - 1153 ETFs with >= 10 holdings (candidates for test set)
 - 162 ETFs with < 10 holdings

Target split ratios: Train=0.70, Validation=0.10, Test=0.20
Target set sizes (approx): Train=921, Validation=131, Test=263

Assigning Test Set (target: 263 ETFs, source: >= 10 holdings group)...
  Assigned 263 ETFs to the Test set.

Assigning Train/Validation Sets from remaining 1052 ETFs...
  Targeting Train=921, Validation=131 from the remaining pool.

Final assigned counts: Train=921, Validation=131, Test=263
  Total assigned ETFs match total unique ETFs (1315).

Actual split ratios: Train=70.04%, Validation=9.96%, Test=20.00%

Creating final DataFrames...

Train DataFrame shape:      (115039, 8)
Validation DataFrame shape: (13395, 8)
Test DataFrame shape:       (28275, 8)

Total rows in split: 156709
Row count matches original DataFrame size (156709).

Verification successful: All 263 ETFs in the test set have >= 10 holdings.


TOTAL ETF 1000
10 Holding 900
TRAIN 900 * 0.8
TEST 900 * 0.2 = 180

In [6]:
train_stage1_origin = train_df[['etf_desc', 'holding_desc']].copy()
train_stage1_origin.rename(columns={"etf_desc": "anchor", "holding_desc": "positive"}, inplace=True)

valid_stage1_origin = valid_df[['etf_desc', 'holding_desc']].copy()
valid_stage1_origin.rename(columns={"etf_desc": "anchor", "holding_desc": "positive"}, inplace=True)

stage1_train_origin_dataset = Dataset.from_pandas(train_stage1_origin.reset_index(drop=True))
stage1_valid_origin_dataset = Dataset.from_pandas(valid_stage1_origin.reset_index(drop=True))

stage1_origin = DatasetDict({
    "train": stage1_train_origin_dataset,
    "valid": stage1_valid_origin_dataset,
})

stage1_origin.push_to_hub("LUcowork/stage1-original", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/116 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/424 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/LUcowork/stage1-original/commit/1a63a45c30a68469a20cd4edc614da1aa8614a32', commit_message='Upload dataset', commit_description='', oid='1a63a45c30a68469a20cd4edc614da1aa8614a32', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/LUcowork/stage1-original', endpoint='https://huggingface.co', repo_type='dataset', repo_id='LUcowork/stage1-original'), pr_revision=None, pr_num=None)

In [7]:
train_stage1_rewrt = train_df[['etf_desc', 'holding_rewritten']].copy()
train_stage1_rewrt.rename(columns={"etf_desc": "anchor", "holding_rewritten": "positive"}, inplace=True)

valid_stage1_rewrt = valid_df[['etf_desc', 'holding_rewritten']].copy()
valid_stage1_rewrt.rename(columns={"etf_desc": "anchor", "holding_rewritten": "positive"}, inplace=True)

stage1_train_rewrt_dataset = Dataset.from_pandas(train_stage1_rewrt.reset_index(drop=True))
stage1_valid_rewrt_dataset = Dataset.from_pandas(valid_stage1_rewrt.reset_index(drop=True))

stage1_rewrt = DatasetDict({
    "train": stage1_train_rewrt_dataset,
    "valid": stage1_valid_rewrt_dataset,
})

stage1_rewrt.push_to_hub(
    "LUcowork/stage1-rewritten",
    private=True
)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/116 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/14 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/425 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/LUcowork/stage1-rewritten/commit/3327f24b59833c1765aa30f9c7378b64dd9ab739', commit_message='Upload dataset', commit_description='', oid='3327f24b59833c1765aa30f9c7378b64dd9ab739', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/LUcowork/stage1-rewritten', endpoint='https://huggingface.co', repo_type='dataset', repo_id='LUcowork/stage1-rewritten'), pr_revision=None, pr_num=None)

In [8]:
candidate_df = pd.concat([train_df, valid_df], ignore_index=True)
candidate_dataset = Dataset.from_pandas(candidate_df.reset_index(drop=True))

test_dataset = Dataset.from_pandas(test_df.reset_index(drop=True))

eval_data = DatasetDict({
    "candidate": candidate_dataset,
    "test": test_dataset,
})

eval_data.push_to_hub("LUcowork/etf-eval", private=True)

Uploading the dataset shards:   0%|          | 0/2 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/65 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/65 [00:00<?, ?ba/s]

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/29 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/LUcowork/etf-eval/commit/48e81d07dfd9e0099d4f8de0642cbbb1137876c4', commit_message='Upload dataset', commit_description='', oid='48e81d07dfd9e0099d4f8de0642cbbb1137876c4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/LUcowork/etf-eval', endpoint='https://huggingface.co', repo_type='dataset', repo_id='LUcowork/etf-eval'), pr_revision=None, pr_num=None)

In [6]:
query_df = pd.read_csv('./data/queries.csv')
query_df.rename(columns={"index": "query_type"}, inplace=True)

query_dataset = Dataset.from_pandas(query_df.reset_index(drop=True))
query_data = DatasetDict({
    "query": query_dataset
})

query_data.push_to_hub("LUcowork/query-eval", private=True)

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/380 [00:00<?, ?B/s]

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/LUcowork/query-eval/commit/2ecaee774220375cfecce59d54e4df9d823e2389', commit_message='Upload dataset', commit_description='', oid='2ecaee774220375cfecce59d54e4df9d823e2389', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/LUcowork/query-eval', endpoint='https://huggingface.co', repo_type='dataset', repo_id='LUcowork/query-eval'), pr_revision=None, pr_num=None)